In [1]:
from math import pow


class State:

    # hard-coded values for energy costs in hall
    hall_steps = {1: [2, 1, 1, 3, 5, 7, 8], 2: [4, 3, 1, 1, 3, 5, 6], 3: [6, 5, 3, 1, 1, 3, 4], 4: [8, 7, 5, 3, 1, 1, 2]}

    def __init__(self, chambers, energy=0, stop_energy=1000000, depth=2):
        self.chamber_depth = depth
        self.chambers = chambers
        self.energy = energy
        self.stop_energy = stop_energy
        self.position_block, self.chamber_block = self._check_blockes()
        self.chamber_space_free = [False] * 4

    def _check_blockes(self):
        position_block = []
        chamber_block = [True] * 4
        for chamber_n, chamber in enumerate(self.chambers[1:], start = 1):
            position_block.append([False] * self.chamber_depth)
            for j, value in reversed(list(enumerate(chamber))):
                if value == chamber_n:
                    position_block[-1][j] = True
                else:
                    chamber_block[chamber_n -1] = False
                    break
        return position_block, chamber_block

    def get_possible_moves(self):
        total_moves = 0
        total_returned = 0
        for move in self._get_possible_moves():
            total_moves += 1
            energy = self._calc_energy(move)
            if self.energy + energy < self.stop_energy:
                yield move, energy
                total_returned += 1

    def _get_possible_moves(self):
        # now moves from the hallway
        for hall_position, val in enumerate(self.chambers[0]):
            if val is not None:
                # find if route from current to chamber is possible
                if hall_position < val:
                    if all(self.chambers[0][n] is None for n in range(hall_position + 1, val + 1)) and self.chamber_space_free[val - 1]:
                        yield (0, hall_position), (val, self.chamber_space_free[val - 1] - 1)
                else:
                    if all(self.chambers[0][n] is None for n in range(val + 1, hall_position)) and self.chamber_space_free[val - 1]:
                        yield (0, hall_position), (val, self.chamber_space_free[val - 1] - 1)
        # now from chamber
        for chamber_n, chamber in enumerate(self.chambers[1:], start=1):
            if self.chamber_block[chamber_n - 1]:
                continue
            for room_index, val in enumerate(chamber):
                if not self.position_block[chamber_n -1][room_index] and chamber[room_index] is not None:
                    # try to get it in in one go, then don't even try other.
                    if self.chamber_space_free[val - 1]:
                        # check nothing in the hall
                        if val < chamber_n:
                            # to the left
                            if all(self.chambers[0][n] is None for n in range(val + 1, chamber_n + 1)):
                                yield (chamber_n, room_index), (val, self.chamber_space_free[val - 1] - 1)
                        else:
                            #to the right
                            if all(self.chambers[0][n] is None for n in range(chamber_n + 1, val + 1)):
                                yield (chamber_n, room_index), (val, self.chamber_space_free[val - 1] - 1)
                    else:
                        yield from self._try_move_left(chamber_n, room_index)
                        yield from self._try_move_right(chamber_n, room_index)
                    break

    def is_solved(self):
            if all(self.chamber_block):
                return self.energy + 1
            return False

    def _try_move_left(self, chamber, start_pos):
        start = chamber
        while start >= 0:
            if self.chambers[0][start] is None:
                yield ((chamber, start_pos), (0, start))
            else:
                break
            start -= 1

    def _try_move_right(self, chamber, start_pos):
        start = chamber + 1
        while start <= 6:
            if self.chambers[0][start] is None:
                yield ((chamber, start_pos), (0, start))
            else:
                break
            start += 1

    def _calc_energy(self, move):
        # always from chamber to hall, otherwise reverse
        # two parts chamber into hall and part in hall
        val = self.chambers[move[0][0]][move[0][1]]
        if move[0][0] != 0 and move[1][0] != 0:
            start_chamber, start_pos, end_chamber, end_pos = move[0][0], move[0][1], move[1][0], move[1][1]
            inside_hall_steps = abs(end_chamber - start_chamber) * 2
            into_hall_steps = start_pos + 1 + end_pos + 1
        else:
            if move[0][0] == 0:
                # start in hall, thus switch
                start_chamber, start_pos, hall_pos = move[1][0], move[1][1], move[0][1]
            else:
                start_chamber, start_pos, hall_pos = move[0][0], move[0][1], move[1][1]
            into_hall_steps = start_pos + 1
            inside_hall_steps = self.hall_steps[start_chamber][hall_pos]
        return (inside_hall_steps + into_hall_steps) * pow(10, val - 1)

    def make_move(self, move, energy):
        self.chambers[move[1][0]][move[1][1]], self.chambers[move[0][0]][move[0][1]] = self.chambers[move[0][0]][move[0][1]], None
        self.energy += energy
        # update blocked position, chambers blocked and space free
        if move[1][0] > 0:
            self.position_block[move[1][0] - 1][move[1][1]] = True
            if move[1][1] == 0:
                self.chamber_block[move[1][0]-1] = True
            self.chamber_space_free[move[1][0] - 1] = move[1][1]
        if move[0][0] > 0:
            # if you go out and the rest is blocked there is space free.
            if all(self.position_block[move[0][0] - 1][move[0][1] + 1:]):
                self.chamber_space_free[move[0][0] - 1] = self.chamber_depth - sum(self.position_block[move[0][0] - 1])


    def undo_move(self, move, energy):
        self.energy -= energy
        self.chambers[move[0][0]][move[0][1]], self.chambers[move[1][0]][move[1][1]] = self.chambers[move[1][0]][move[1][1]], None
        # update blocked position, chambers blocked and space free
        if move[1][0] > 0:
            self.position_block[move[1][0] - 1][move[1][1]] = False
            self.chamber_block[move[1][0] - 1] = False
            self.chamber_space_free[move[1][0] - 1] = move[1][1] + 1
        if move[0][0] > 0:
            # if you go out and the rest is blocked there is no space free.
            if all(self.position_block[move[0][0] - 1][move[0][1] + 1:]):
                self.chamber_space_free[move[0][0] - 1] = 0

    def __str__(self):
        return str(self.chambers)

In [2]:
test_chambers = [[None] * 7, [2, 1], [3, 4], [2, 3], [4, 1]]
start_state = State(test_chambers)
start_state.is_solved()
print(start_state.chamber_space_free, start_state.chamber_block, start_state.position_block)

[False, False, False, False] [False, False, False, False] [[False, True], [False, False], [False, True], [False, False]]


In [3]:
def progress(state, moves, current_val=1000000):
    start_state.stop_energy = current_val
    for move, energy in start_state.get_possible_moves():
        moves.append(move)
        state.make_move(move, energy)
        if state.is_solved():
            val = state.is_solved() - 1
        else:
            val = progress(state, current_val=current_val, moves=moves)
        if val < current_val:
            current_val = val
            state.stop_energy = val
        state.undo_move(move, energy)
        moves.pop()
    return current_val

In [4]:
input_chambers = [[None] * 7, [1, 3], [4, 4], [1, 2], [3, 2]]
start_state = State(input_chambers, depth=2)
progress(start_state, moves=[])

15385.0

In [5]:
input_chambers = [[None] * 7, [1,4, 4, 3], [4,3, 2, 4], [1,2, 1, 2], [3,1, 3, 2]]
start_state = State(input_chambers, depth=4)
progress(start_state, moves=[], current_val=52_000)

49803.0